<a href="https://colab.research.google.com/github/Raniamea/arabic-video-summarisation/blob/main/notebooks/01_transcribe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🚨 Reset Colab Python Kernel Automatically After Downgrading NumPy
!pip install numpy==1.23.5 --force-reinstall --no-cache-dir
import os; os.kill(os.getpid(), 9)

In [ ]:
# Install Whisper and Torch
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch==2.0.1 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
!apt-get install ffmpeg
!pip install -q pydub

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import os

# Path to your params.json on Google Drive
param_path = "/content/drive/MyDrive/ArabicVideoSummariser/params.json"

# Load it
with open(param_path, "r") as f:
    params = json.load(f)

# Get the filename
video_filename = params.get("video_file")
print("🎥 Transcribing video file:", video_filename)

from pydub import AudioSegment
import math

# Define base paths
base_path = "/content/drive/MyDrive/ArabicVideoSummariser"
videos_path = os.path.join(base_path, "videos")
transcripts_path = os.path.join(base_path, "transcripts")

video_path = os.path.join(videos_path, video_filename)
video_name = os.path.splitext(video_filename)[0]
transcript_path = os.path.join(transcripts_path, f"{video_name}_ar.txt")
translation_path = os.path.join(transcripts_path, f"{video_name}_en.txt")
trascription_json_path = os.path.join(transcripts_path, f"{video_name}_ar.json")
translation_json_path = os.path.join(transcripts_path, f"{video_name}_en.json")

# Convert video to audio
audio_path = os.path.join(videos_path, f"{video_name}.wav")
!ffmpeg -y -i "{video_path}" -ar 16000 -ac 1 "{audio_path}"  # Resample to 16kHz mono

# Load audio using pydub
audio = AudioSegment.from_wav(audio_path)
chunk_length_ms = 30 * 1000  # 30 seconds
total_chunks = math.ceil(len(audio) / chunk_length_ms)

print(f"🔊 Audio duration: {len(audio) / 1000:.1f}s, Chunks: {total_chunks}")


In [ ]:
import torch, whisper, json, gc

torch.cuda.empty_cache()
gc.collect()

model = whisper.load_model("large", device="cuda", in_memory=True)


results = []
for i in range(total_chunks):
    start_ms = i * chunk_length_ms
    end_ms = min((i + 1) * chunk_length_ms, len(audio))
    chunk = audio[start_ms:end_ms]
    chunk_file = f"/content/chunk_{i}.wav"
    chunk.export(chunk_file, format="wav")

    print(f"⏱️ Transcribing chunk {i+1}/{total_chunks} ({start_ms/1000:.1f}s - {end_ms/1000:.1f}s)")
    result = model.transcribe(chunk_file, language="ar", task="transcribe", verbose=False, fp16=False)

    for segment in result['segments']:
        segment["start"] += start_ms / 1000
        segment["end"] += start_ms / 1000
        results.append(segment)

# Save text transcript
with open(transcript_path, "w", encoding="utf-8") as f:
    f.write(" ".join([seg["text"] for seg in results]))

# Save time-coded transcript
with open(transcript_path.replace(".txt", "_with_timecodes.txt"), "w", encoding="utf-8") as f:
    for seg in results:
        f.write(f"[{seg['start']:.2f} - {seg['end']:.2f}] {seg['text']}\n")

# Save JSON
with open(trascription_json_path, "w", encoding="utf-8") as f:
    json.dump({"segments": results}, f, ensure_ascii=False, indent=2)


In [ ]:
del model
torch.cuda.empty_cache()
gc.collect()